# Applied Data Science Capstone - Segment & Cluster

# Scraping Data From Wikipedia

In [249]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')
#table_rows

# Converting Data Into a Pandas Dataframe

In [250]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

In [251]:
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


# Removing Boroughs That Are Not Assigned

In [252]:
df = df[df.Borough != 'Not assigned']
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [253]:
df[df.Borough == 'Not assigned']

,PostalCode,Borough,Neighbourhood


# Replacing Neighbourhoods That Are Not Assigned To Equal Borough

In [254]:
df[df.Neighbourhood == 'Not assigned']

,PostalCode,Borough,Neighbourhood
9,M7A,Queen's Park,Not assigned


In [255]:
df = df.replace(to_replace ="Not assigned", 
                 value ="Queen\'s Park")

# Check That All Boroughs and Neighbourhoods Are Assigned

In [256]:
df[df.Borough == 'Not assigned']

,PostalCode,Borough,Neighbourhood


In [257]:
df[df.Neighbourhood == 'Not assigned']

,PostalCode,Borough,Neighbourhood


# Merging Neighbourhoods With The Same PostalCode

In [258]:
df[df.PostalCode == 'M5A']

,PostalCode,Borough,Neighbourhood
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park


In [259]:
df[df.Neighbourhood == 'Harbourfront']

,PostalCode,Borough,Neighbourhood
5,M5A,Downtown Toronto,Harbourfront


In [260]:
df[df.Neighbourhood == 'Regent Park']

,PostalCode,Borough,Neighbourhood
6,M5A,Downtown Toronto,Regent Park


In [261]:
df = df[df.Neighbourhood != 'Harbourfront']

In [262]:
df = df.replace(to_replace ="Regent Park", 
                 value ="Regent Park, Harbourfront")

In [263]:
df[df.PostalCode == 'M5A']

,PostalCode,Borough,Neighbourhood
6,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Displaying The Shape and Final Dataframe

In [264]:
df.shape

(210, 3)

In [265]:
df

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
6,M5A,Downtown Toronto,"Regent Park, Harbourfront"
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern
15,M3B,North York,Don Mills North


# Adding Latitudes and Longitudes To Dataframe

In [274]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data

--2019-06-22 04:25:37--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 159.8.72.228
Connecting to cocl.us (cocl.us)|159.8.72.228|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-22 04:25:37--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-22 04:25:37--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-06-22 04

In [275]:
location = pd.read_csv('Geospatial_Coordinates.csv')
location.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
location.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [277]:
df_location = pd.merge(df,location,on='PostalCode')

In [278]:
df_location

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,Lawrence Heights,43.718518,-79.464763
4,M6A,North York,Lawrence Manor,43.718518,-79.464763
5,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
6,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
7,M1B,Scarborough,Rouge,43.806686,-79.194353
8,M1B,Scarborough,Malvern,43.806686,-79.194353
9,M3B,North York,Don Mills North,43.745906,-79.352188
